In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
from ta.momentum import StochRSIIndicator
from ta.momentum import RSIIndicator
from ta.volume import ForceIndexIndicator
from ta.volume import EaseOfMovementIndicator
from ta.volume import VolumePriceTrendIndicator
from ta.volume import NegativeVolumeIndexIndicator
from ta.trend import SMAIndicator
from ta.trend import DPOIndicator
from ta.volatility import BollingerBands
from ta.volatility import KeltnerChannel
from ta.volatility import AverageTrueRange
from ta.volatility import DonchianChannel
from ta.others import DailyLogReturnIndicator
from ta.momentum import ROCIndicator


payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
#print(first_table.head())
sip=np.array(payload.iloc[:,0])
sip=sip[sip!='BRK.B']
sip=sip[sip!='BF.B']
df=pd.DataFrame()
dfout=pd.DataFrame()
start_date="2014-01-01"
end_date="2021-11-01"
txt = "{:%}"
wind=14
for i in range(0,1):
    try: 
        stk=sip[i]
        #df = yf.download(stk, start_date, end_date,interval=intr).dropna()

        dfd = yf.download(stk, start_date, end_date,interval='1d').dropna()
        dfd['Open1']=dfd.Open.shift(1)
        dfd['Close1']=dfd.Close.shift(1)
        dfd['High1']=dfd.High.shift(1)
        dfd['Low1']=dfd.Low.shift(1)
        dfd['Volume1']=dfd.Volume.shift(1)
        dfd['ChangePre']=dfd.Close-dfd.Close1
        dfd['Change']=dfd.Close-dfd.Open
        dfd['ChangePreP']=dfd.ChangePre/dfd.Close1
        dfd['ChangeP']=dfd.Change/dfd.Open

        dfd['fi']=ForceIndexIndicator(close=dfd.Close1,volume=dfd.Volume1).force_index()
        #dfd['efi_rsid']=RSIIndicator(close=dfd.efid,window=wind,fillna=True).rsi()
        #dfd["kd"]=StochRSIIndicator(close=dfd.Close_1,window=wind,smooth1=3, smooth2=3).stochrsi_k() 
        dfd['bbh']=BollingerBands(close=dfd.Close1).bollinger_hband()
        dfd['dpo']=DPOIndicator(close=dfd.Close1).dpo()
        dfd['kch']=KeltnerChannel(close=dfd.Close1,high=dfd.High1,low=dfd.Low1).keltner_channel_hband() 
        dfd['dch']=DonchianChannel(close=dfd.Close1,high=dfd.High1,low=dfd.Low1).donchian_channel_hband() 
        dfd['bbli']=BollingerBands(close=dfd.Close1).bollinger_lband_indicator()
        dfd['kcli']=KeltnerChannel(close=dfd.Close1,high=dfd.High1,low=dfd.Low1).keltner_channel_lband_indicator() 
        dfd['atr']=AverageTrueRange(high=dfd.High1,low=dfd.Low1,close=dfd.Close1).average_true_range()
        dfd['nvi']=NegativeVolumeIndexIndicator(close=dfd.Close1,volume=dfd.Volume1).negative_volume_index()
        dfd['eom']=EaseOfMovementIndicator(high=dfd.High1,low=dfd.Low1,volume=dfd.Volume1).ease_of_movement()
        dfd['seom']=EaseOfMovementIndicator(high=dfd.High1,low=dfd.Low1,volume=dfd.Volume1).sma_ease_of_movement()
        dfd['vpt']=VolumePriceTrendIndicator(close=dfd.Close1,volume=dfd.Volume1).volume_price_trend()
        dfd['dlr']=DailyLogReturnIndicator(close=dfd.Close1).daily_log_return()
        dfd['ROC']=ROCIndicator(close=dfd.Close1).roc()
        dfd=dfd.dropna()
        x=1
        x2=1

        conditions = [ 
        (dfd['Open']>0)

        #& (dfd.seom<np.percentile(dfd.seom,x))
        #& (dfd.dpo>np.percentile(dfd.dpo,100-x))
        #& (dfd.eom<np.percentile(dfd.eom,x))
        #& (dfd.dlr<np.percentile(dfd.dlr,x))   
        #&(dfd.fi<np.percentile(dfd.fi,x))
        &(dfd.ROC<np.percentile(dfd.ROC,x))
        #& (dfd.vpt<np.percentile(dfd.vpt,x))
        


        ,
        (dfd['Open']>0)
        #& (dfd.seom>np.percentile(dfd.seom,100-x))
        #& (dfd.dpo<np.percentile(dfd.dpo,x))
        #& (dfd.fi>np.percentile(dfd.fi,100-x))
        & (dfd.ROC>np.percentile(dfd.ROC,100-x))
        #& (dfd.eom>np.percentile(dfd.eom,100-x2))
        #& (dfd.vpt>np.percentile(dfd.vpt,100-x))
        #& (dfd.dlr>np.percentile(dfd.dlr,100-x2))

        ]

        choices = [
        1,-1
        ]
        dfd['trade_pass']=np.select(conditions,choices,default=0)
        dfd_long=dfd[dfd['trade_pass']==1]
        dfd_short=dfd[dfd['trade_pass']==-1]
        fir=dfd.iloc[0,0]
        las=dfd.iloc[len(dfd.index)-1,3]

        hold_prof=(las)/(fir)-1
        hold_wr=len(dfd[dfd.ChangePre>0])/(len(dfd.index))
        hold_avg=np.sum(dfd.ChangePreP)/len(dfd.index)
        hold_tr=1

        long_prof=(np.sum(dfd_long.Change)+fir)/fir-1
        long_prof_pre=(np.sum(dfd_long.ChangePre)+fir)/fir-1
        long_wr=len(dfd_long[dfd_long.Change>0])/(len(dfd_long.index))
        long_wr_pre=len(dfd_long[dfd_long.ChangePre>0])/(len(dfd_long.index))
        long_ave=np.sum(dfd_long.ChangeP)/len(dfd_long.index)
        long_ave_pre=np.sum(dfd_long.ChangePreP)/len(dfd_long.index)   
        long_tr=len(dfd_long.index)/(len(dfd.index))


        short_prof=(-np.sum(dfd_short.Change)+fir)/fir-1
        short_prof_pre=(-np.sum(dfd_short.ChangePre)+fir)/fir-1
        short_wr=len(dfd_short[dfd_short.Change<0])/(len(dfd_short.index))
        short_wr_pre=len(dfd_short[dfd_short.ChangePre<0])/(len(dfd_short.index))
        short_ave=(-np.sum(dfd_short.ChangeP))/len(dfd_short.index)
        short_ave_pre=(-np.sum(dfd_short.ChangePreP))/len(dfd_short.index)   
        short_tr=len(dfd_short.index)/(len(dfd.index))




        temp=pd.Series([stk,
                        hold_prof,hold_wr,hold_avg,hold_tr,
                        long_prof,long_wr,long_ave,long_tr,
                        long_prof_pre,long_wr_pre,long_ave_pre,long_tr,
                        short_prof,short_wr,short_ave,short_tr,
                        short_prof_pre,short_wr_pre,short_ave_pre,short_tr])
        dfout=dfout.append(temp,ignore_index=True)
    except Exception as e: print(e)

print(dfout)
fin1=np.array([])
fin2=np.array([])
fin3=np.array([])
fin4=np.array([])
fin5=np.array([])
for j in range (1,5):
    temp1=(np.average(dfout.iloc[:,j]))    
    fin1=np.append(fin1,temp1)
for j in range (5,9):
    temp2=(np.average(dfout.iloc[:,j]))    
    fin2=np.append(fin2,temp2)
for j in range (9,13):
    temp3=(np.average(dfout.iloc[:,j]))    
    fin3=np.append(fin3,temp3)
for j in range (13,17):
    temp4=(np.average(dfout.iloc[:,j]))    
    fin4=np.append(fin4,temp4)
for j in range (17,21):
    temp5=(np.average(dfout.iloc[:,j]))    
    fin5=np.append(fin5,temp5)   
    
    
    
    
dff1=pd.DataFrame(fin1.reshape(1,-1))
dff2=pd.DataFrame(fin2.reshape(1,-1))
dff3=pd.DataFrame(fin3.reshape(1,-1))
dff4=pd.DataFrame(fin4.reshape(1,-1))
dff5=pd.DataFrame(fin5.reshape(1,-1))




#print(dff1)
#print(dff2)
dff=pd.concat([dff1,dff2,dff3,dff4,dff5])
dff.columns=(['total%','win_rate%','avg%/trade','trade rate'])
dff.index=(['hold','long-day','long-overnight','short-day','short-overnight'])
#dff=pd.DataFrame(fin.reshape(1,-1))
#dff.columns=(['hold_prof','long_prof','long_prof_pre','long_wr','long_wr_pre','long_ave','long_ave_pre','trade_rate_long'])
print(dff)
dff.to_csv('bt.csv')

[*********************100%***********************]  1 of 1 completed
    0         1         2         3    4         5     6         7         8   \
0  MMM  0.411374  0.534324  0.000269  1.0  0.071327  0.55  0.003277  0.010246   

         9   ...        11        12        13    14        15        16  \
0  0.136335  ...  0.006989  0.010246 -0.040679  0.35 -0.001918  0.010246   

         17   18        19        20  
0  0.005845  0.4 -0.000291  0.010246  

[1 rows x 21 columns]
                   total%  win_rate%  avg%/trade  trade rate
hold             0.411374   0.534324    0.000269    1.000000
long-day         0.071327   0.550000    0.003277    0.010246
long-overnight   0.136335   0.500000    0.006989    0.010246
short-day       -0.040679   0.350000   -0.001918    0.010246
short-overnight  0.005845   0.400000   -0.000291    0.010246
